# Advanced System Calls with Popen

This section is an old lecture which teaches system calls using the __subprocess.Popen__ class. Making system calls with __subprocess.check_output()__ is far simpler, and so we have opted to teach the __check_output()__ function instead. However, this function is simply a wrapper around the (much more versitile) __Popen__ class. We are leaving this supplementary lecture here for any students who are ready for a look under the hood. While we don't get into it explicitly, in this lecture you will start to see the object-oriented aspect of Python.

## subprocess.Popen
The core of the subprocess module is the **Popen** class, which is a running child process object.

In [3]:
import subprocess as sp
 
child = sp.Popen(['ls', '-l'])
returncode = child.wait()

print child
print 'process finished with return code', returncode

process finished with return code 0


Here, we used Popen to run the command **ls -l**. But we don't see the output. That's because the Jupyter notebook only displays the output from the Python interpreter process, not the output of it's child processes. If you copy and paste this code into a python script and run it from a terminal you will see the output of the **ls** command written to your terminal, all jumbled up with the output from the Python interpreter.

You'll notice that **Popen** takes one mandatory argument, a list of command line arguments that would be separated by spaces spaces on the terminal. So **ls -a -1 /home/james** becomes **['ls', '-a', '-1', '/home/james']**.

### *Popen.*wait()
The **wait()** method of the **Popen** object tells the system to wait until this subprocess has completed before continuing. After the subprocess finishes, **wait()** also returns the return code of the program, which is 0 if the program completed without error, and any other number (usually 1) if it closed with an error.

**wait()** has two very imortant uses. First, you may need to wait for the output of the subprocess before your next step. Try this code:

In [ ]:
import subprocess as sp

# touch updates the timestamp of a file,
# or, if the file doesn't exist, creates it.
child = sp.Popen(['touch', 'testfile'])
child.wait()
fh = open('testfile', 'r')
fh.read()
fh.close()    

Now run it again.

You'll notice that the first time it gave you an error message because 'testfile' did not exist yet, even though you had just started the subprocess to create it. The second time 'testfile' already exists and you don't get an error.

Use this code to remove 'testfile' and try it again:

In [ ]:
child = sp.Popen(['rm', 'testfile'])

The second important use for **wait()** is to prevent your script from hogging all of your system resources. If you start running a ton of subprocesses at once it can prevent you from doing anything else, be rude to other users on the server, and is also sometimes less efficient then letting one command run to completion before starting the next.

## Capturing Output
The above example might be useful for running gzip or Tophat or some other program that creates it's own output files, but what if we want to store the output of a program like **ls** that writes to standard out? We could tell the Popen object to write the output to a file, or we could capture it in a *pipe*.

Let's test how to do this by asking Python to do something very meta: run another Python script! Here's the script we will be running, which should be saved in your 5.2 directory as 'test_output.py'.

In [6]:
%%writefile test_output.py
#!/usr/bin/env python
import sys
sys.stdout.write('this is standard output')
sys.stderr.write('this is standard error')

Overwriting test_output.py


### Redirecting Output to Filehandles
To redirect the output from a Popen object to a file, pass **Popen** a filehandle like so:

In [ ]:
import subprocess as sp

# open files to print to
outfh = open('out.txt', 'w')
errfh = open('err.txt', 'w')

process = sp.Popen(['python', 'test_output.py'], stdout=outfh, stderr=errfh)
process.wait()

outfh.close()
errfh.close()

Check 'out.txt' and 'err.txt' to make sure the two outputs went where you expected them.

### *subprocess.*PIPE
But we often want to do something with the output of a program, either parsing it and reformating it, continuing the analysis in Python, or turning it into a figure. To do this, we will *pipe* the output directly into Python's memory.

A *pipe* is used to send the output of one program into the input of another. You have already used them on the Unix command line with the **|** character. For example: **ls -1 | grep '.fastq' | wc -l** will count the number of files containing '.fastq' in the current directory.

Of course, the syntax is different in Python. . .

In [4]:
import subprocess as sp
 
child = sp.Popen(['python', 'test_output.py'], stdout=sp.PIPE, stderr=sp.PIPE)

# we use the communicate() method to get both the stdout and the stderr
output, error = child.communicate()

print "communicate() returned a tuple of two strings."

print "------------------------"
print "The first string is the stdout:"
print output

print "------------------------"

print "The second string is the stderr:"
print error
print "------------------------"
print

# And it is easy to convert the output into a friendlier structure
output = output.strip().split()
print "And here's stdout converted into a nice Python list:"
print output

communicate() returned a tuple of two strings.
------------------------
The first string is the stdout:
this is a test

------------------------
The second string is the stderr:

------------------------

And here's stdout converted into a nice Python list:
['this', 'is', 'a', 'test']


The **communicate()** method does several things. First, it waits for the process to complete, similar to **wait()**. Next it returns both output streams as a two item tuple. If we were to write **communicate()** as a function it would look a little like this:

In [ ]:
def communicate(self):
    self.wait()
    out = self.stdout.read()
    err = self.stderr.read()
    return out, err

proc = sp.Popen(['python', 'test_output.py'], stdout=sp.PIPE, stderr=sp.PIPE)
print communicate(proc)

However, Popen's **communicate()** is a bit smarter than this. *Pipes* are an input/output buffer with a limited size. The fill up with data when they are written to, and then they are emptied of data when they are read from, kind of like, you know, actual pipes. **wait()** can therefore *block*, waiting for the pipe to be emptied before the program can continue. Imagine flushing the toilet if the other end wasn't draining - you would get an overflow - and that's exactly what can happen here.

### Piping input with **communicate()**
**communicate()** can also be used to pass input into a program.

In [ ]:
shakes = """'Double, double toil and trouble;
    Fire burn, and caldron bubble.
    Fillet of a fenny snake,
    In the caldron boil and bake;
    Eye of newt, and toe of frog,
    Wool of bat, and tongue of dog,
    Adder's fork, and blind-worm's sting,
    Lizard's leg, and owlet's wing,—
    For a charm of powerful trouble,
    Like a hell-broth boil and bubble.
    Double, double toil and trouble;
    Fire burn, and caldron bubble."""

# count the lines, words, and characters in this passage
proc = sp.Popen(['wc', '-l'], stdout=sp.PIPE, stderr=sp.PIPE, stdin=sp.PIPE)
print proc.communicate(input=shakes)

In [ ]:
# Find the path of the users home directory.
env = sp.Popen(['env'], stdout=sp.PIPE)
grep = sp.Popen(['grep', 'HOME='], stdin=env.stdout, stdout=sp.PIPE)
out, err = grep.communicate()

home = out.split('=')[1]
print home

## BLAST
Good ol'e Basic Local Alignment Search Tool, now that's bioinformatics even Grandma can get down with. In its most basic form, BLAST takes a short sequence of either amino acids or nucleotides, searches a database of reference sequences, and returns the most likely matches for the query sequence. It's brilliant, and the only trouble is that we're often interested in BLASTing hella query sequences, and much less interested in the mundanity of parsing the output that comes back to us. Fortunately, we can easily script the execution of BLAST such that we can run queries many times with different sequences and settings, and also parse the output back into dictionaries to manipulate and even store longterm on the disk.

The first part of this section will show us how to install a new program in our UNIX environment, and the second part will let us practice using **Popen** to script the execution of BLAST.

### Installing BLAST
I suspect most everyone here is accustomed to using the NCBI BLAST web interface, but it's just another program, whether it's running on an NCBI web server, or on our laptops. First we will install BLAST locally, such that we have ready access to it from anywhere on our computers.

In the PythonCourse/ProgramFiles directory, there is a tar.gz file containing the BLAST executable for your platform. The .tar.gz file extension tells us that the file has been archived (the .tar part) and compressed (the .gz part). The UNIX command to uncompress and expand the file is:

**tar -zxvf [filename]**

And we can tell **tar** where to dump the output with the **-C** option, like so:

**tar -zxvf [filename] -C [output directory]**

Within the BLAST folder, We've got a ChangeLog file that will tell us what version of BLAST we've got, a doc directory with documentation, a data directory with the various matrices to use for esimating substituion rates, and then the *bin* directory. *bin* stands for binary, and directories named *bin* are conventionally used to store executable binary files. Try typing **which less** on a terminal and you will see that **less**, just like all your terminal commands (**ls**, **mv**, **cat**, etc), is actually an executable binary, probably in a directory like */usr/bin*. 

### Creating a BLAST Database

Okay, now that we have these program available, let's start using them.

Since BLAST is going to compare a query sequence to a database of sequences, collected from say, a genome, or a group of genomes, we first need to create the reference database to BLAST against. Many of these are available for download, but we're going to create our own from the S. cerevisiae genome.

The yeast genome is available on the NCBI FTP server, though we have included it in your course folder. 'yeastn.nt' is the fasta formated genome, while 'yeast.aa' is the fasta formated collection of all proteins.

To make your own local BLAST database, use the** makeblaskdb** program, found within the BLAST folder

```bash
~/Dropbox/2017_Summer_Python/ProgramFiles/linux/ncbi-blast-2.2.26+/bin/makeblastdb -in yeast.nt -out yeast_genome -dbtype nucl
```

Now we have a fully operational BLAST installation and a yeast nucleotide database to search against. We can see that BLAST has created some additional files for our yeast database in our working directory (.nhr, .nin, and .nsq).

### Running BLAST!

Now let's query the database with blastn, piping our query sequence directly into the program.

In [ ]:
import subprocess as sp

testseq = 'TGTTGTATTACGGGCTCGAGTAATACCGGAGTGTCTTGACAATCCTAATATAAA' \
          'CGGTCTTAGGGAAGTAACCAGTTGTCAAAACAGTTTATCAGATTAATTCACGGA' \
          'ATGTTACTTATCTTATATATTATATAAAATATGAtATCATATTAAGTGGTGGAA' \
          'GCGCGGAATCTCGGATCTAAACTAATTGTTCAGGCATTTATACTTTTGGTAGTT' \
          'CAGCTAGGGAAGGACGGGTTTTATCTCATGTTGTTCGTTTTGTTATAAGGTTGT' \
          'TTCATATGTGTTTTATGAACGTTTAGGATGACGTATTGTCATACTGACATATCT' \
          'CATTTTGAGATACAACA'

##This has to be set manually.
blastBin ='/home/james/Dropbox/2017_Summer_Python/ProgramFiles/linux/ncbi-blast-2.2.26+/bin/' 


def blastn(querySeq):
    program = blastBin + 'blastn'
    database = 'yeast_genome'
    proc = sp.Popen([program, '-db', database], stdout=sp.PIPE, stderr=sp.PIPE, stdin=sp.PIPE)
    out, err = proc.communicate(input=testseq)
    return out

print blastn(testseq)

You can imagine how you would set a list of query sequences, or a series of databases to query against, or even vary some of the BLAST settings (for example, the minimum E-value to report), and run this again. You also might care which species your hit came from (if you were searching against a larger database), or which chromosome hits were on (to determine synteny, e.g.)

In this case, we've used the PIPE to capture the output, which might be helpful when we want to systematically sort through the results in each BLAST hit, only storing the results in certain circumstances. However, we can also write the results to an output file instead of the PIPE. Let's try that:

In [ ]:
import subprocess as sp

testseq = 'ATGGTTCATTTAGGTCCAAAGAAACCACAGGCTAGAAAGGGTTCCATGGCTGATGTGCCCAAGGAATTGATGGATGAAAT' \
        'TCATCAGTTGGAAGATATGTTTACAGTTGACAGCGAGACCTTGAGAAAGGTTGTTAAGCACTTTATCGACGAATTGAATA' \
        'AAGGTTTGACAAAGAAGGGAGGTAACATTCCAATGATTCCCGGTTGGGTCATGGAATTCCCAACAGGTAAAGAATCTGGT' \
        'AACTATTTGGCCATTGATTTGGGTGGTACTAACTTAAGAGTCGTGTTGGTCAAGTTGAGCGGTAACCATACCTTTGACAC' \
        'CACTCAATCCAAGTATAAACTACCACATGACATGAGA'

##This has to be set manually.
blastBin ='/home/james/Dropbox/2017_Summer_Python/ProgramFiles/linux/ncbi-blast-2.2.26+/bin/' 

        
def blastn(querySeq, outfh):
    program = blastBin + 'blastn'
    database = 'yeast_genome'
    proc = sp.Popen([program, '-db', database], stdout=outfh, stderr=sp.PIPE, stdin=sp.PIPE)
    out, err = proc.communicate(input=testseq)
    return out

fh = open('blastn_output.txt', 'w')
blastn(testseq, fh)
fh.close()

**Windows Note:**

On Windows piping the input to blastn doesn't work.  However, you can call it by outputing the sequence to a file, like this:

```python
with open('seq_test.txt', 'w') as fout:
    fout.write(testseq);
```

And then calling blastn like this instead:

```python
proc = sp.Popen([program, '-db', database, '-query', 'seq_test.txt'], stdout=outfh)
```